In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 150

## Not sure why this isn't working

The straight replace is working. The partial one isn't.

In [2]:
def clean_country_and_code(df):
    df['country_codes'].replace('XXX', np.nan, inplace=True)
    df['country_codes'].str.replace(';XXX|XXX;', '')
    df['country_codes'] = df.loc[df['country_codes'].notnull(), 'country_codes'].str.split(';').apply(lambda x: sorted(x))
    df['countries'].replace('Not identified', np.nan, inplace=True)
    df['countries'].str.replace(';Not identified|Not identified;', '')
    df['countries'] = df.loc[df['countries'].notnull(), 'countries'].str.split(';').apply(lambda x: sorted(x))

In [3]:
keep_cols = ['node_id', 'sourceID', 'name', 'countries', 'country_codes']
dtypes = {'node_id': 'int32', 'sourceID':'category', 'name':'object',
            'countries':'category', 'country_codes':'category'}

bahamas_intermediary_raw = pd.read_csv('../data/raw/bahamas_leaks/bahamas_leaks.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
bahamas_intermediary_raw = bahamas_intermediary_raw[keep_cols]

offshore_intermediary_raw = pd.read_csv('../data/raw/offshore_leaks/offshore_leaks.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
offshore_intermediary_raw = offshore_intermediary_raw[keep_cols]

panama_intermediary_raw = pd.read_csv('../data/raw/panama_papers/panama_papers.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
panama_intermediary_raw = panama_intermediary_raw[keep_cols]

paradise_intermediary_raw = pd.read_csv('../data/raw/paradise_papers/paradise_papers.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
paradise_intermediary_raw = paradise_intermediary_raw[keep_cols]

### To Do

- There are companies and people in the name field. Do you want to break this into different tables? One with specific people and one with companies?
- Break peope into first and last name?
- Companies are broken out by their branch office. Do you want to make a Corporate column that unites these offices?

In [4]:
intermediary_df = pd.concat([bahamas_intermediary_raw, offshore_intermediary_raw, panama_intermediary_raw, paradise_intermediary_raw], ignore_index=True)
intermediary_df.head()

,node_id,sourceID,name,countries,country_codes
0,23000001,Bahamas Leaks,Internal User,NaN,NaN
1,23000002,Bahamas Leaks,M & A CORPORATE SERVICES LIMITED,Bahamas,BHS
2,23000003,Bahamas Leaks,BRITANNIA CONSULTING GROUP,Bahamas,BHS
3,23000004,Bahamas Leaks,ABACO FSC LTD.,Bahamas,BHS
4,23000005,Bahamas Leaks,OL PRIVATE CORPORATE COUNSEL LTD.,Bahamas,BHS


In [18]:
intermediary_df['sourceID'].value_counts()

Panama Papers                                    14110
Offshore Leaks                                    9526
Paradise Papers - Barbados corporate registry      974
Bahamas Leaks                                      541
Paradise Papers - Bahamas corporate registry       239
Paradise Papers - Appleby                          185
Paradise Papers - Nevis corporate registry          96
Paradise Papers - Aruba corporate registry          74
Name: sourceID, dtype: int64

In [12]:
print(intermediary_df[intermediary_df['name'].duplicated(keep=False)].sort_values('name').shape, '\n')
print(intermediary_df[intermediary_df['name'].duplicated(keep=False)].sort_values('name').nunique(), '\n')
intermediary_df[intermediary_df['name'].duplicated(keep=False)].sort_values('name').head()

(1706, 5) 

node_id          1706
sourceID            7
name              583
countries          61
country_codes      61
dtype: int64 



,node_id,sourceID,name,countries,country_codes
1402,135063428,Paradise Papers - Barbados corporate registry,(BARBADOS) INC. SG GLOBAL CONSULTING,NaN,NaN
1254,135126973,Paradise Papers - Barbados corporate registry,(BARBADOS) INC. SG GLOBAL CONSULTING,NaN,NaN
607,135020603,Paradise Papers - Barbados corporate registry,(BARBADOS) INC. SG GLOBAL CONSULTING,NaN,NaN
1164,135075791,Paradise Papers - Barbados corporate registry,(BARBADOS) LTD. AMICORP,NaN,NaN
971,135116404,Paradise Papers - Barbados corporate registry,(BARBADOS) LTD. AMICORP,NaN,NaN


In [14]:
print(intermediary_df[intermediary_df['name'].str.title().duplicated(keep=False)].sort_values('name').shape, '\n')
print(intermediary_df[intermediary_df['name'].str.title().duplicated(keep=False)].sort_values('name').nunique(), '\n')
intermediary_df[intermediary_df['name'].str.title().duplicated(keep=False)].sort_values('name').tail()

(2193, 5) 

node_id          2193
sourceID            8
name             1070
countries          69
country_codes      69
dtype: int64 



,node_id,sourceID,name,countries,country_codes
9121,297421,Offshore Leaks,YenBle Management Limited,Taiwan,TWN
9134,297440,Offshore Leaks,Yew Su-Sun,Hong Kong,HKG
9135,297441,Offshore Leaks,Yew Su-Sun,Hong Kong,HKG
3567,11003568,Panama Papers,ZETLAND CORPORATE SERVICES LIMITED,Hong Kong,HKG
2990,291134,Offshore Leaks,Zetland Corporate Services Limited,Hong Kong,HKG


In [15]:
for col in ['countries', 'country_codes']:
    print(col)
    print(intermediary_df[col].value_counts(), '\n')
    
# countries = Not Identified. There are some in lists. You'll need to get rid of those too
# country_codes = XXX. There are some in lists. You'll need to get rid of those too

countries
Hong Kong                                              4895
United Kingdom                                         1545
United States                                          1510
Taiwan                                                 1323
Switzerland                                            1321
Singapore                                              1307
Bahamas                                                 897
China                                                   646
Panama                                                  559
Indonesia                                               522
Guatemala                                               441
Luxembourg                                              403
Brazil                                                  402
Not identified                                          394
Thailand                                                358
Malaysia                                                350
Ecuador                       

In [5]:
intermediary_df['name'] = intermediary_df['name'].str.title()
intermediary_df.head()

,node_id,sourceID,name,countries,country_codes
0,23000001,Bahamas Leaks,Internal User,NaN,NaN
1,23000002,Bahamas Leaks,M & A Corporate Services Limited,Bahamas,BHS
2,23000003,Bahamas Leaks,Britannia Consulting Group,Bahamas,BHS
3,23000004,Bahamas Leaks,Abaco Fsc Ltd.,Bahamas,BHS
4,23000005,Bahamas Leaks,Ol Private Corporate Counsel Ltd.,Bahamas,BHS


In [16]:
intermediary_df.loc[intermediary_df['countries'].str.contains(';', na=False), 'countries'].value_counts()

Jersey;United Kingdom                                  310
Guernsey;United Kingdom                                236
Panama;United States                                    12
Hong Kong;China                                          6
United States;Venezuela                                  4
Macao;China                                              4
Panama;Mexico                                            4
United States;Panama                                     4
Panama;Switzerland                                       3
United States;Colombia                                   3
Panama;Spain                                             3
China;Hong Kong                                          3
Singapore;Hong Kong                                      3
Switzerland;United Kingdom                               3
Singapore;Not identified                                 3
Panama;Brazil                                            3
Switzerland;Monaco                                      

In [17]:
intermediary_df.loc[intermediary_df['country_codes'].str.contains(';', na=False), 'country_codes'].value_counts()

JEY;GBR            310
GGY;GBR            236
PAN;USA             12
HKG;CHN              6
PAN;MEX              4
USA;PAN              4
USA;VEN              4
MAC;CHN              4
USA;COL              3
SGP;HKG              3
SGP;XXX              3
CHN;HKG              3
PAN;CHE              3
PAN;ESP              3
PAN;BRA              3
CHE;GBR              3
XXX;SGP              2
XXX;USA              2
XXX;IDN              2
CYP;LUX              2
USA;XXX              2
PAN;CAN              2
CHE;MCO              2
PAN;GTM              2
MNE;SRB              2
USA;ARG              2
GBR;LUX              2
GBR;MCO              2
PAN;DEU              2
XXX;MYS              2
IDN;XXX              2
THA;HKG              2
CHE;LUX              1
ARE;CYP              1
RUS;XXX              1
ARG;BHS              1
VGB;USA              1
CHE;FRA              1
XXX;AUS              1
USA;SAU              1
USA;DOM              1
SGP;CYM              1
COL;USA              1
TZA;MUS    

In [5]:
clean_country_and_code(intermediary_df)

In [6]:
intermediary_df['countries'].value_counts()

[Hong Kong]                                                 4895
[United Kingdom]                                            1545
[United States]                                             1510
[Taiwan]                                                    1323
[Switzerland]                                               1321
[Singapore]                                                 1307
[Bahamas]                                                    897
[China]                                                      646
[Panama]                                                     559
[Indonesia]                                                  522
[Guatemala]                                                  441
[Luxembourg]                                                 403
[Brazil]                                                     402
[Thailand]                                                   358
[Malaysia]                                                   350
[Ecuador]                

In [7]:
intermediary_df['country_codes'].value_counts()

[HKG]                   4895
[GBR]                   1545
[USA]                   1510
[TWN]                   1323
[CHE]                   1321
[SGP]                   1307
[BHS]                    897
[CHN]                    646
[PAN]                    559
[IDN]                    522
[GTM]                    441
[LUX]                    403
[BRA]                    402
[THA]                    358
[MYS]                    350
[ECU]                    323
[GBR, JEY]               310
[URY]                    296
[CAN]                    236
[GBR, GGY]               236
[IMN]                    230
[COL]                    227
[ESP]                    203
[AUS]                    201
[VEN]                    182
[ARE]                    179
[CYP]                    176
[MCO]                    170
[ISR]                    146
[RUS]                    137
[FRA]                    124
[LIE]                    123
[CRI]                    121
[KNA]                    102
[DEU]         

In [ ]:
intermediary_df.to_csv('../data/intermediate/intermediary.csv')